In [3]:
# ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [1]:
# ConversationBufferWindowMemory
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output) :
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)
add_message(5, 5)

memory.load_memory_variables({})

In [2]:
# ConversationSummaryMemory
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output) :
    memory.save_context({"input": input}, {"output": output})

def get_history() :
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go")
get_history()

# {'history': 'The human introduces themselves as Nicolas and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'}

In [3]:
# ConversationSummaryBufferMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output) :
    memory.save_context({"input": input}, {"output": output})

def get_history() :
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
get_history()

In [1]:
# ConversationKGMemory
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

def add_message(input, output) :
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is Nicolas"})
# {'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}


add_message("Nicolas likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"input": "what does nicolas like"})

# {'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}

In [14]:
# Memory on LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = '''
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You:
'''

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is Chan")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: My name is Chan
    You:


> Finished chain.


'Hello Chan! How can I assist you today?'

In [2]:
# Chat Based Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpul AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# template = '''
#     You are a helpful AI talking to a human.

#     {chat_history}
#     Human: {question}
#     You:
# '''

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="My name is Chan")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpul AI talking to a human
Human: My name is Chan

> Finished chain.


'Hello Chan! How can I assist you today?'

In [3]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpul AI talking to a human
Human: My name is Chan
AI: Hello Chan! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"That's great, Chan! Seoul is a vibrant and bustling city. How can I help you with anything related to Seoul or any other topic you have in mind?"

In [4]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpul AI talking to a human
Human: My name is Chan
AI: Hello Chan! How can I assist you today?
Human: I live in Seoul
AI: That's great, Chan! Seoul is a vibrant and bustling city. How can I help you with anything related to Seoul or any other topic you have in mind?
Human: What is my name?

> Finished chain.


'Your name is Chan.'

In [12]:
# LCEL Based Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpul AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(input) :
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question) :
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

In [13]:
invoke_chain("My name is Chan")

content='Hello Chan! How can I assist you today?'


In [14]:
invoke_chain("What is my name?")

content='Your name is Chan.'


In [1]:
# LCEL Based Memory
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpul AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(input) :
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question) :
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

In [2]:
invoke_chain("My name is Chan")

content='Hello Chan! How can I assist you today?'


In [3]:
invoke_chain("What is my name?")

content='Your name is Chan.'
